In [25]:
import pinecone
import pandas as pd
#pinecone.init(api_key=open("Pineconekey.txt","r").read(), environment="gcp-starter")
#pc = pinecone(api_key=open("Pineconekey.txt","r").read(), environment="gcp-starter")   
import os
from pinecone import Pinecone, ServerlessSpec, PodSpec

pc = Pinecone(api_key=open("Pineconekey.txt","r").read())


In [8]:
pc.delete_index("alice")

In [9]:
pc.create_index(
  name="alice",
  dimension=3072,
  metric="cosine",
  spec=PodSpec(
    environment="gcp-starter",
    pod_type="starter",
    pods=1
  ))

index = pc.Index("alice")
print(pc.list_indexes())

{'indexes': [{'dimension': 3072,
              'host': 'alice-a980bfb.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'alice',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [10]:
import pandas as pd
import numpy as np

datafile_path = "data/Alice_with_embeddings.csv"
df = pd.read_csv(datafile_path)
#df["embedding"] = df.embedding.apply(eval).apply(np.array)
df["embedding"] = df["embedding"].apply(eval)  # Convert string to Python list
df["embedding"] = df["embedding"].apply(lambda arr: np.array(arr).tolist())  # Convert NumPy array to Python list

df = df.rename(columns={"Unnamed: 0": "id", "embedding": "values", "paragraph": "paragraph"})

dfvalues = df.copy()
dfvalues.drop("paragraph", axis=1, inplace=True)
dfvalues['id'] = dfvalues['id'].astype(str)
dfvalues.head()

,id,values
0,0,"[0.025997638702392578, -0.025250762701034546, ..."
1,1,"[0.004169656429439783, 0.016068601980805397, -..."
2,2,"[-0.01431865431368351, -0.005995984189212322, ..."
3,3,"[0.03022223338484764, 0.04146571084856987, -0...."
4,4,"[0.016702160239219666, 0.025803163647651672, -..."


In [11]:
index.upsert_from_dataframe(dfvalues[0:200])
index.upsert_from_dataframe(dfvalues[201:400])
index.upsert_from_dataframe(dfvalues[401:600])
index.upsert_from_dataframe(dfvalues[601:800])
index.upsert_from_dataframe(dfvalues[801:1000])
index.upsert_from_dataframe(dfvalues[1001:1200])


sending upsert requests:   0%|          | 0/200 [00:00<?, ?it/s]

sending upsert requests:   0%|          | 0/199 [00:00<?, ?it/s]

sending upsert requests:   0%|          | 0/199 [00:00<?, ?it/s]

sending upsert requests:   0%|          | 0/199 [00:00<?, ?it/s]

sending upsert requests:   0%|          | 0/15 [00:00<?, ?it/s]

sending upsert requests: 0it [00:00, ?it/s]

{'upserted_count': 0}

In [12]:
from time import sleep

ready = False
while not ready:
    try:
        if pc.describe_index('alice')["status"]['ready']:
            ready = True
            print('Index is ready!')
        else:
            print('Index is not ready yet...')
            sleep(5)
    except pc.core.client.exceptions.NotFoundException:
        # NotFoundException means the index is created yet.
        pass
        

Index is ready!


In [13]:
index.describe_index_stats()

{'dimension': 3072,
 'index_fullness': 0.00812,
 'namespaces': {'': {'vector_count': 812}},
 'total_vector_count': 812}

In [17]:
def searchdatabase(searchterm = 'Is Alice Happy?', results = 3):

    #Get the embedding of the searchterm
    import openai
    openai.api_type = "azure"
    openai.api_base = "https://eastusOpenAIral.openai.azure.com/"
    openai.api_version = "2022-12-01"
    openai.api_key = open("AzurekeyEUS.txt","r").read()

    from openai.embeddings_utils import get_embedding, cosine_similarity
    query_embedding = get_embedding(searchterm,engine="textembeddings3large")

    ##Get the IDs
    xc = index.query(vector=query_embedding,
    top_k=results,
    include_values=False)
    c = [x["id"] for x in xc["matches"]]
    dfres = pd.DataFrame(c, columns=['id'])
    dfres['id'] = dfres['id'].astype(str)
    dfres

    ### join the IDs back to get the text
    dfres['id'] = dfres['id'].astype(int)
    df['id'] = df['id'].astype(int)
    dffinalSearch = dfres.join(df, how='left', on='id',  lsuffix='_left', rsuffix='_right')

    return dffinalSearch['paragraph'].to_list()


In [15]:
index = pc.Index("alice")

In [18]:
searchdatabase('What can you eat in wonderland?', 5)

["'I've had nothing yet,' Alice replied in an offended tone, 'so I can't\ntake more.'",
 'The Caterpillar and Alice looked at each other for some time in silence:\nat last the Caterpillar took the hookah out of its mouth, and addressed\nher in a languid, sleepy voice.',
 'On this the White Rabbit blew three blasts on the trumpet, and then\nunrolled the parchment scroll, and read as follows:--',
 "However, this bottle was NOT marked 'poison,' so Alice ventured to taste\nit, and finding it very nice, (it had, in fact, a sort of mixed flavour\nof cherry-tart, custard, pine-apple, roast turkey, toffee, and hot\nbuttered toast,) she very soon finished it off.",
 "Soon her eye fell on a little glass box that was lying under the table:\nshe opened it, and found in it a very small cake, on which the words\n'EAT ME' were beautifully marked in currants. 'Well, I'll eat it,' said\nAlice, 'and if it makes me grow larger, I can reach the key; and if it\nmakes me grow smaller, I can creep under the 

In [31]:
#Get the embedding of the searchterm
import openai
openai.api_type = "azure"
openai.api_base = "https://southcentralOpenAIral.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = open("AzurekeySCUS.txt","r").read()
from openai.embeddings_utils import get_embedding, cosine_similarity
query_embedding = get_embedding("WHat can you eat in wonderland?",engine="textembeddingada002v2")

#query the database
index.query(vector=query_embedding,
top_k=15,
include_values=False)

{'matches': [{'id': '388', 'score': 0.8250615, 'values': []},
             {'id': '151', 'score': 0.8171857, 'values': []},
             {'id': '25', 'score': 0.806956, 'values': []},
             {'id': '26', 'score': 0.802425802, 'values': []},
             {'id': '17', 'score': 0.801261604, 'values': []},
             {'id': '569', 'score': 0.798997343, 'values': []},
             {'id': '637', 'score': 0.797648787, 'values': []},
             {'id': '542', 'score': 0.797089219, 'values': []},
             {'id': '143', 'score': 0.79491359, 'values': []},
             {'id': '362', 'score': 0.793453038, 'values': []},
             {'id': '794', 'score': 0.793062031, 'values': []},
             {'id': '227', 'score': 0.792041123, 'values': []},
             {'id': '408', 'score': 0.790407419, 'values': []},
             {'id': '144', 'score': 0.790087283, 'values': []},
             {'id': '16', 'score': 0.789170146, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}